# ISYE 6501 Homework #11

## Jeremy Wong | kwong301@gatech.edu

# Question 15.2

In the videos, we saw the "diet problem". (The diet problem is one of the first large-scale optimization problems to be studied in practice. Back in the 1930's and 40's, the Army wanted to meet the nutritional requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file `diet.xls`. 

1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using `PuLP`.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)
2.	Please add to your model the following constraints (which might require adding more variables) and solve the new model:
        *	(a). If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
        *	(b). Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
        *	(c). To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]

If you want to see what a more full-sized problem would look like, try solving your models for the file `diet_large.xls`, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake).  I don’t know anyone who’d want to eat this diet – the optimal solution includes dried chrysanthemum garland, raw beluga whale flipper, freeze-dried parsley, etc. – which shows why it’s necessary to add additional constraints beyond the basic ones we saw in the video!
	    
[Note: there are many optimal solutions, all with zero cholesterol, so you might get a different one.  It probably won’t be much more appetizing than mine.]

# Answers to 15.2 (1)

Denote the index of food and nutrient by $x$ and $k$. Let $p_x$, $q_x$ be the price and quantity for food $x$. Let $n_x^k$ be the per unit nutrient $k$, where $m^k$, $M^k$ are the daily intake lower and upper bounds. We can write the linear programming problem as:

\begin{align*}
& \min_{\{q_x\}} \sum_x p_x q_x \text{ subject to }\\
& \sum_x q_x n^k_x  \geq m^k \\
& \sum_x q_x n^k_x \leq M^k \\
& q_x \geq 0 \text{ for all } x 
\end{align*}

We will use `PuLP` to solve this linear programming. Check [here](https://www.coin-or.org/PuLP/pulp.html) for documentation and [here](https://github.com/tirthajyoti/Optimization-Python/blob/master/Balanced_Diet_Problem_Complex.ipynb) for a great example for usage. The solution is:

```python
Portion_Celery,_Raw = 52.64371
Portion_Frozen_Broccoli = 0.25960653
Portion_Lettuce,Iceberg,Raw = 63.988506
Portion_Oranges = 2.2929389
Portion_Poached_Eggs = 0.14184397
Portion_Popcorn,Air_Popped = 13.869322
The total cost of the optimal meal is 4.34
```

# Code for 15.2 (1)

In [61]:
import pandas as pd
import pulp

df0 = pd.read_excel('./hw11/diet.xls', header=0)
df = df0.iloc[0:63]
min_intake, max_intake = df0.iloc[65], df0.iloc[66]

food_items = df['Foods'].tolist()
nutrient_items = df.columns[3:].tolist()

mincost_prob = pulp.LpProblem("Simple Diet Problem", pulp.LpMinimize)
q_x = pulp.LpVariable.dicts(name="Portion", indexs=food_items, lowBound=0, cat='Continuous')

df.set_index('Foods', inplace=True)
mincost_prob += pulp.lpSum([df.loc[i, 'Price/ Serving'] * q_x[i] for i in food_items]), "Total Cost of the balanced diet"
for n in nutrient_items:
    mincost_prob += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) >= min_intake[n], '{}Minimum'.format(n.split(' ')[0])
    mincost_prob += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) <= max_intake[n], '{}Maximum'.format(n.split(' ')[0])
    
mincost_prob

Simple_Diet_Problem:
MINIMIZE
0.23*Portion_2%_Lowfat_Milk + 0.16*Portion_3.3%_Fat,Whole_Milk + 0.24*Portion_Apple,Raw,W_Skin + 0.16*Portion_Apple_Pie + 0.16*Portion_Bagels + 0.15*Portion_Banana + 0.15*Portion_Bologna,Turkey + 0.05*Portion_Butter,Regular + 0.31*Portion_Cap'N_Crunch + 0.07*Portion_Carrots,Raw + 0.04*Portion_Celery,_Raw + 0.25*Portion_Cheddar_Cheese + 0.28*Portion_Cheerios + 0.39*Portion_Chicknoodl_Soup + 0.03*Portion_Chocolate_Chip_Cookies + 0.28*Portion_Corn_Flks,_Kellogg'S + 0.39*Portion_Couscous + 0.65*Portion_Crm_Mshrm_Soup,W_Mlk + 0.27*Portion_Frankfurter,_Beef + 0.16*Portion_Frozen_Broccoli + 0.18*Portion_Frozen_Corn + 0.32*Portion_Grapes + 0.33*Portion_Ham,Sliced,Extralean + 0.83*Portion_Hamburger_W_Toppings + 0.31*Portion_Hotdog,_Plain + 0.15*Portion_Kielbasa,Prk + 0.49*Portion_Kiwifruit,Raw,Fresh + 0.02*Portion_Lettuce,Iceberg,Raw + 0.17*Portion_Macaroni,Ckd + 0.52*Portion_Malt_O_Meal,Choc + 0.99*Portion_New_E_Clamchwd,W_Mlk + 0.75*Portion_Neweng_Clamchwd + 0.82

In [77]:
mincost_prob.solve()
print("Status:", pulp.LpStatus[mincost_prob.status])

for v in mincost_prob.variables():
    if v.varValue > 0 and v.name[0]=='P':
        print(v.name, "=", v.varValue)
        
print('The total cost of the optimal meal is {:.2f}'.format(mincost_prob.objective.value()))

Status: Optimal
Portion_Celery,_Raw = 52.64371
Portion_Frozen_Broccoli = 0.25960653
Portion_Lettuce,Iceberg,Raw = 63.988506
Portion_Oranges = 2.2929389
Portion_Poached_Eggs = 0.14184397
Portion_Popcorn,Air_Popped = 13.869322
The total cost of the optimal meal is 4.34


# Answers to 15.2 (2a)

Solve the above problem with the additional constraint: If a food is selected, then a minimum of 1/10 serving must be chosen. So the problem becomes:

\begin{align*}
& \min_{\{q_x\}} \sum_x p_x q_x \text{ subject to }\\
& \sum_x q_x n^k_x  \geq m^k \\
& \sum_x q_x n^k_x \leq M^k \\
& q_x \leq 0.1 I\{q_x > 0\} \\
& q_x \geq 0 \text{ for all } x \\
\end{align*}

Here $I\{X\}$ represents the indicator for condition $X$, it is equal to 1 is $X$ is true, and 0 otherwise. Essentially, the additional constraints requires that if food $x$ is chosen in the optimal solution, then the quantity $q_x$ is required to be at least 0.1. However, this will induce non-linearity to the problem. Instead, we define an integer variable $i_x$ indicating whether food $x$ was chosen:

\begin{align*}
& \min_{\{q_x\}} \sum_x p_x q_x \text{ subject to }\\
& \sum_x q_x n^k_x  \geq m^k \\
& \sum_x q_x n^k_x \leq M^k \\
& 0.1 i_x \leq q_x \leq B i_x \\
& q_x \geq 0 \text{ for all } x \\
& i_x = 0 \text{ or } 1 
\end{align*}

Here an arbitrary large number $B$ is introduced to bound $q_x$ to ensure that when $x$ is chosen $i_x$ will indeed be 1. We see that the solution in the previous question satisfies the minimum portion constraint. So we expect the same solution. Solving the linear programming problem with `PuLP` we can confirm this:

```python
Portion_Celery,_Raw = 52.64371
Portion_Frozen_Broccoli = 0.25960653
Portion_Lettuce,Iceberg,Raw = 63.988506
Portion_Oranges = 2.2929389
Portion_Poached_Eggs = 0.14184397
Portion_Popcorn,Air_Popped = 13.869322
The total cost of the optimal meal is 4.34
```

# Code to 15.2 (2a)

In [84]:
mincost_prob_2a = pulp.LpProblem("Simple Diet Problem", pulp.LpMinimize)
q_x = pulp.LpVariable.dicts(name="Portion", indexs=food_items, lowBound=0, cat='Continuous')
i_x = pulp.LpVariable.dicts(name="Chosen", indexs=food_items, lowBound=0, upBound=1,cat='Integer')

mincost_prob_2a += pulp.lpSum([df.loc[i, 'Price/ Serving'] * q_x[i] for i in food_items]), "Total Cost of the balanced diet"
for n in nutrient_items:
    mincost_prob_2a += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) >= min_intake[n], '{}Minimum'.format(n.split(' ')[0])
    mincost_prob_2a += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) <= max_intake[n], '{}Maximum'.format(n.split(' ')[0])

for f in food_items:
    mincost_prob_2a += q_x[f] >= 0.1 * i_x[f], '{}_MinimumPortion'.format(f)
    mincost_prob_2a += q_x[f] <= 1e6 * i_x[f], '{}_Regularity'.format(f)

mincost_prob_2a.solve()
print("Status:", pulp.LpStatus[mincost_prob_2a.status])

for v in mincost_prob_2a.variables():
    if v.varValue > 0 and v.name[0]=='P':
        print(v.name, "=", v.varValue)
        
print('The total cost of the optimal meal is {:.2f}'.format(mincost_prob_2a.objective.value()))

Status: Optimal
Portion_Celery,_Raw = 52.64371
Portion_Frozen_Broccoli = 0.25960653
Portion_Lettuce,Iceberg,Raw = 63.988506
Portion_Oranges = 2.2929389
Portion_Poached_Eggs = 0.14184397
Portion_Popcorn,Air_Popped = 13.869322
The total cost of the optimal meal is 4.34


# Answers to 15.2 (2b)

Solve the above problem with the additional constraint: At most one of frozon broccoli and celery can be chosen. The problem becomes:

\begin{align*}
& \min_{\{q_x\}} \sum_x p_x q_x \text{ subject to }\\
& \sum_x q_x n^k_x  \geq m^k \\
& \sum_x q_x n^k_x \leq M^k \\
& i_b + i_c \leq 1\\
& 0.1 i_x \leq q_x \leq B i_x \\
& q_x \geq 0 \text{ for all } x \\
& i_x = 0 \text{ or } 1 
\end{align*}

We see that the solution now requires a slightly higher total cost, because the additional constraint is binding:

```python
Portion_Celery,_Raw = 43.154119
Portion_Lettuce,Iceberg,Raw = 80.919121
Portion_Oranges = 3.0765161
Portion_Peanut_Butter = 2.0464575
Portion_Poached_Eggs = 0.14184397
Portion_Popcorn,Air_Popped = 13.181772
The total cost of the optimal meal is 4.49
```

# Code to 15.2 (2b)

In [108]:
mincost_prob_2b = pulp.LpProblem("Simple Diet Problem", pulp.LpMinimize)

mincost_prob_2b += pulp.lpSum([df.loc[i, 'Price/ Serving'] * q_x[i] for i in food_items]), "Total Cost of the balanced diet"
for n in nutrient_items:
    mincost_prob_2b += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) >= min_intake[n], '{}Minimum'.format(n.split(' ')[0])
    mincost_prob_2b += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) <= max_intake[n], '{}Maximum'.format(n.split(' ')[0])

for f in food_items:
    mincost_prob_2b += q_x[f] >= 0.1 * i_x[f], '{}_MinimumPortion'.format(f)
    mincost_prob_2b += q_x[f] <= 1e6 * i_x[f], '{}_Regularity'.format(f)

mincost_prob_2b += i_x['Frozen Broccoli'] + i_x['Celery, Raw'] <= 1, 'Celery_v_Broccoli'

mincost_prob_2b.solve()
print("Status:", pulp.LpStatus[mincost_prob_2b.status])

for v in mincost_prob_2b.variables():
    if v.varValue > 0 and v.name[0]=='P':
        print(v.name, "=", v.varValue)
        
print('The total cost of the optimal meal is {:.2f}'.format(mincost_prob_2b.objective.value()))

Status: Optimal
Portion_Celery,_Raw = 43.154119
Portion_Lettuce,Iceberg,Raw = 80.919121
Portion_Oranges = 3.0765161
Portion_Peanut_Butter = 2.0464575
Portion_Poached_Eggs = 0.14184397
Portion_Popcorn,Air_Popped = 13.181772
The total cost of the optimal meal is 4.49


# Answers to 15.2 (2c)

Solve the above problem with the additional constraint: To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. In regards to "variety", __I understand this statement as: we need at least 3 types of meat/poultry/fish/eggs in diet. So "Scrambled Eggs"+"Poached Eggs"+"Pork" is not OK because there are just two types - eggs and meat__. 

Now let $g$ index each of the group and $r_g$ be the indicator that any food in group $g$ is chosen, the problem becomes:

\begin{align*}
& \min_{\{q_x\}} \sum_x p_x q_x \text{ subject to }\\
& \sum_x q_x n^k_x  \geq m^k \\
& \sum_x q_x n^k_x \leq M^k \\
& b r_g \leq \sum_{x \in g} i_x \leq Br_g \\
& \sum_g r_g \geq 3 \\
& i_b + i_c \leq 1\\
& 0.1 i_x \leq q_x \leq B i_x \\
& q_x \geq 0 \text{ for all } x \\
& i_x, r_x = 0 \text{ or } 1 \\
\end{align*}

Again, we introduce an arbitrarily large number $B$ to ensure that when any of the relevant items is chosen, the group indicator $r_g$ will indeed equal to 1. We also impose a small number $b < 1$ so that when none of the relevant items in group $g$ is chosen, the group $r_g$ will be 0. Since the previous solution does not have foods in the meat, fish, and poultry categories, the new constraints will be binding. 

```python
Portion_Bologna,Turkey = 0.1
Portion_Celery,_Raw = 42.003189
Portion_Kielbasa,Prk = 0.1
Portion_Lettuce,Iceberg,Raw = 83.411008
Portion_Oranges = 3.0862592
Portion_Peanut_Butter = 1.9541879
Portion_Poached_Eggs = 0.12033097
Portion_Popcorn,Air_Popped = 13.233318
The total cost of the optimal meal is 4.52
```

Indeed, we see from below that the new solution is the old solution plus minimal amount of cheapest protein options that are already not included. The current solution has poultry (Turkey Bologna), meat (Prok Kielbasa), and eggs (Poached Eggs).

In [122]:
cat_items = {'meat_items': ['Frankfurter, Beef', 'Ham,Sliced,Extralean', 'Kielbasa,Prk', 
                            'Hamburger W/Toppings', 'Hotdog, Plain', 'Pork'],
             'egg_items': ['Poached Eggs', 'Scrambled Eggs'],
             'fish_items': ['Sardines in Oil', 'White Tuna in Water'],
             'poultry_items': ['Roasted Chicken', 'Bologna,Turkey']}

protein_items = []
for g in cat_items.keys():
    protein_items += cat_items[g]

df.loc[protein_items, ['Price/ Serving']].sort_values(by='Price/ Serving')

,Price/ Serving
Foods,
Poached Eggs,0.08
Scrambled Eggs,0.11
"Kielbasa,Prk",0.15
"Bologna,Turkey",0.15
"Frankfurter, Beef",0.27
"Hotdog, Plain",0.31
"Ham,Sliced,Extralean",0.33
Sardines in Oil,0.45
White Tuna in Water,0.69


In [111]:
mincost_prob_2c = pulp.LpProblem("Simple Diet Problem", pulp.LpMinimize)

r_g = pulp.LpVariable.dicts(name="Chosen", indexs=cat_items.keys(), lowBound=0, upBound=1, cat='Integer')

mincost_prob_2c += pulp.lpSum([df.loc[i, 'Price/ Serving'] * q_x[i] for i in food_items]), "Total Cost of the balanced diet"
for n in nutrient_items:
    mincost_prob_2c += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) >= min_intake[n], '{}Minimum'.format(n.split(' ')[0])
    mincost_prob_2c += pulp.lpSum([df.loc[i, n] * q_x[i] for i in food_items]) <= max_intake[n], '{}Maximum'.format(n.split(' ')[0])

for f in food_items:
    mincost_prob_2c += q_x[f] >= 0.1 * i_x[f], '{}_MinimumPortion'.format(f)
    mincost_prob_2c += q_x[f] <= 1e6 * i_x[f], '{}_Regularity'.format(f)
    
mincost_prob_2c += i_x['Frozen Broccoli'] + i_x['Celery, Raw'] <= 1, 'Celery_v_Broccoli'

for g in cat_items.keys():
    mincost_prob_2c += pulp.lpSum([i_x[f] for f in cat_items[g]]) <= 1e6 * r_g[g], '{}_UpCategory'.format(g)
    mincost_prob_2c += pulp.lpSum([i_x[f] for f in cat_items[g]]) >= 1e-6 * r_g[g], '{}_LowCategory'.format(g)

mincost_prob_2c += pulp.lpSum([r_g[g] for g in cat_items.keys()]) >= 3, 'CategoryMinimum'
    
mincost_prob_2c.solve()
print("Status:", pulp.LpStatus[mincost_prob_2c.status])

for v in mincost_prob_2c.variables():
    if v.varValue > 0 and v.name[0]=='P':
        print(v.name, "=", v.varValue)
        
print('The total cost of the optimal meal is {:.2f}'.format(mincost_prob_2c.objective.value()))

Status: Optimal
Portion_Bologna,Turkey = 0.1
Portion_Celery,_Raw = 42.003189
Portion_Kielbasa,Prk = 0.1
Portion_Lettuce,Iceberg,Raw = 83.411008
Portion_Oranges = 3.0862592
Portion_Peanut_Butter = 1.9541879
Portion_Poached_Eggs = 0.12033097
Portion_Popcorn,Air_Popped = 13.233318
The total cost of the optimal meal is 4.52
